In [1]:
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import collections
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('gujarati-train.csv')

In [3]:
def count_length():
    data['word_count'] = data['headline'].apply(lambda x: len(str(x).split(" ")))

In [4]:
count_length()

In [5]:
data.head(10)

,headline,Label,word_count
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports,10
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports,10
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports,10
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11
6,"IND vs WI: ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી...",Sports,15
7,IPL 2022: હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુ...,Sports,12
8,IPL 2022ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ ...,Sports,17
9,IPL 2022: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે...,Sports,12


In [6]:
import re
def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text

In [7]:
for i in range(len(data)):
    data['headline'][i] = processText(data['headline'][i])

In [8]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
data['headline'] = data['headline'].apply(lambda x: tokenization(x))

In [9]:
for i in range(len(data)):
    data['headline'][i] = [s.replace("\n", "") for s in data['headline'][i]]

In [10]:
f = open("Gujarati_stopwords.txt", "r",encoding="utf8")
stopwords_gu = []
for i in f:
    i=i.split("\n")
    stopwords_gu.append(str(i[0]))

In [11]:
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
to_be_removed = stopwords_gu + punctuations + stopwords_en

In [12]:
for i in range(len(data)):
    data['headline'][i]=[ele for ele in data['headline'][i] if ele not in (to_be_removed)]
count_length()
data.head(10)

,headline,Label,word_count
0,"[ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલાડીઓની...",Sports,6
1,"[રોહિત, કોહલીના, વિવાદ, ગાવસ્કરે, કર્યો, મોટો,...",Sports,10
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, સ્...",Sports,7
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,10
4,"[ક્યાં, રમાશે, મેચ, સૌરવ, ગાંગુલીએ, જવાબ]",Sports,6
5,"[ટીમ, ઈન્ડિયાનો, ખેલાડી, વેડફી, રહ્યો, ટેલેન્ટ...",Sports,9
6,"[ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, મેચ, નહી, ...",Sports,10
7,"[હરાજીમાં, ખેલાડીઓનું, વેચાવવું, મુશ્કેલ, કરોડ...",Sports,7
8,"[મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, વિદેશી, ખે...",Sports,10
9,"[ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, મેગા,...",Sports,8


In [13]:
corpus_list =[]
for i in range(len(data)):
    corpus_list +=data['headline'][i]
counter=collections.Counter(corpus_list)

In [14]:
least_common= [word for word, word_count in Counter(corpus_list).most_common()[:-50:-1]]
for i in range(len(data)):
    data['headline'][i]=[ele for ele in data['headline'][i] if ele not in (least_common)]

In [15]:
data['headline'] = data['headline'].str.join(" ")
data.head(10)

,headline,Label,word_count
0,ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓની બોલી,Sports,6
1,રોહિત કોહલીના વિવાદ ગાવસ્કરે કર્યો મોટો ખુલાસો...,Sports,10
2,પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ સ્પિડથી ...,Sports,7
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,10
4,ક્યાં રમાશે મેચ સૌરવ ગાંગુલીએ જવાબ,Sports,6
5,ટીમ ઈન્ડિયાનો ખેલાડી વેડફી રહ્યો ટેલેન્ટ ગાવસ્...,Sports,9
6,ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ મેચ નહી રમી શકે...,Sports,10
7,હરાજીમાં ખેલાડીઓનું વેચાવવું મુશ્કેલ કરોડોમાં ...,Sports,7
8,મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે વિદેશી ખેલાડી કર...,Sports,10
9,ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં મેગા ઓક્શનમાં,Sports,8


In [16]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data,test_size=0.15, random_state=42)

In [17]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB(alpha = 1))
model.fit(train.headline, train.Label)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB(alpha=1))])

In [18]:
predicted_categories = model.predict(test.headline)

In [19]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print("The accuracy is {}".format(accuracy_score(test.Label, predicted_categories)*100))

The accuracy is 76.72489853538028


In [20]:
mat = confusion_matrix(test.Label, predicted_categories)
mat

array([[782,  83,  43,  68, 144],
       [ 49, 925,  46,  50,  46],
       [ 55,  65, 929,  62,  35],
       [ 58, 115,  89, 872,  57],
       [118,  70,  33,  33, 840]], dtype=int64)

In [21]:
report = classification_report(test.Label, predicted_categories)
print(report)

               precision    recall  f1-score   support

     Business       0.74      0.70      0.72      1120
Entertainment       0.74      0.83      0.78      1116
     Politics       0.81      0.81      0.81      1146
       Sports       0.80      0.73      0.77      1191
         Tech       0.75      0.77      0.76      1094

     accuracy                           0.77      5667
    macro avg       0.77      0.77      0.77      5667
 weighted avg       0.77      0.77      0.77      5667



In [22]:
import joblib
import os
loaded_model = joblib.dump(model, 'GUJ_nb_final.pkl')

In [23]:
simple_test = ["PAKનો 5 વિકેટથી વિજય:રસાકસી બાદ અફઘાનિસ્તાને 19મી ઓવરમાં મેચ ગુમાવી, આસિફ અલીએ એક ઓવરમાં 4 સિક્સ મારી પાક.ને મેચ જિતાડીયો"]
pred1 = model.predict(simple_test)
pred1[0]

'Sports'

In [24]:
simple_test2 = ["શેરમાર્કેટમાં કડાકો:10 દિવસમાં Mcap 15 લાખ કરોડ ઘટ્યું; સેન્સેક્સ 678 પોઇન્ટ તૂટ્યો, ઇન્વેસ્ટર્સે 3 દિવસમાં 6.15 લાખ ગુમાવ્યાં"]
pred2 = model.predict(simple_test2)
pred2[0]

'Business'

In [25]:
simple_test3 = ["SRKના દીકરાની ઘરવાપસી LIVE:આર્યન ખાનને 'મન્નત'થી આર્થર રોડ જેલનું 13 કિલોમીટરનું અંતર કાપતાં 28 દિવસ થયા"]
pred3 = model.predict(simple_test3)
pred3[0]

'Entertainment'

In [26]:
simple_test4 = ["લોન્ચ થયો Oppo નો Waterproof સ્માર્ટફોન, ધાંસૂ કેમેરા સાથે મળશે ગજબના ફીચર્સ"]
pred4 = model.predict(simple_test4)
pred4[0]

'Tech'

In [27]:
simple_test5 = ["ઉત્તરાખંડમાં ભાજપને મોટો ઝટકો, કેબિનેટ મંત્રી પદેથી રાજીનામું આપીને આ નેતા કોંગ્રેસમાં થયા સામેલસ"]
pred5 = model.predict(simple_test5)
pred5[0]

'Business'